In [ ]:
#
# Which two actors starred the most together across the dataset?
# ... or two movies are the most similar (based on actors in common)?
#
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
import json
import time
import os
from rtsvg import *
rt = RACETrack()
ofi = rt.ontologyFrameworkInstance(base_filename='../../../data/kaggle_imdb_600k/20240519_ontology')

In [ ]:
#
# 2m31s - amd7900x | edges 1,973,892 | nodes 1,201,864
#

# Just the cast member relationship
df       = ofi.df_triples.filter(pl.col('vrb') == 'castMemberOf')
df_edges = df.group_by(['sbj', 'obj']).len()

# Create a bipartite graph
B = nx.Graph()
B.add_nodes_from(df_edges['sbj'], bipartite=0)
B.add_nodes_from(df_edges['obj'], bipartite=1)
for i in range(len(df_edges)):
    B.add_edge(df_edges['sbj'][i], df_edges['obj'][i])

# Project the graph onto the 'u' node set
P = nx.bipartite.projected_graph(B, df_edges['sbj'])

# Calculate Jaccard similarity on the projected graph
similarity = nx.jaccard_coefficient(P)

In [ ]:
# Print the top 100
as_list = []
k = 100
for x in similarity:
    i0, i1, sim = x
    as_list.append(x)
    print(f'"{ofi.labeling_uids[i0]}" "{ofi.labeling_uids[i1]}" {sim:0.3f}')
    k -= 1
    if k <= 0: break